In [91]:
import numpy as np
import pandas as pd

import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import pickle


In [ ]:

dataset = pd.read_csv(r"fertiliserNPKdata.csv")
dataset

In [77]:
dataset= dataset.drop(['S(%)'],axis=1)
dataset=dataset.drop(['Mg(%)'],axis=1)
dataset

,Fertiliser,N(%),P(%),K(%)
0,Ammonium chloride,25.0,0,0
1,Ammonium sulphate,20.5,0,0
2,Anhydrous ammonia,82.2,0,0
3,Calcium ammonium nitrate,25.0,0,0
4,Urea,46.0,0,0
5,Ammonium phosphate,20.0,20,0
6,Diammonium phosphate (DAP),16.0,46,0
7,Double super phosphate,0.0,32,0
8,Rock phosphate,0.0,30,0
9,Single super phosphate,0.0,16,0


In [78]:
labels = dataset['Fertiliser'].unique().tolist()
labels.sort()
labels

['Ammonium chloride',
 'Ammonium phosphate',
 'Ammonium sulphate',
 'Anhydrous ammonia',
 'Calcium ammonium nitrate',
 'Diammonium phosphate (DAP)',
 'Double super phosphate',
 'Muriate of potash (MOP)',
 'Potassium magnesium sulphate',
 'Potassium nitrate',
 'Potassium polyphosphate',
 'Rock phosphate',
 'Single super phosphate',
 'Sulphate of potash',
 'Triple super phosphate',
 'Urea']

In [79]:
le = LabelEncoder()
le.fit(dataset['Fertiliser'])
dataset['Fertiliser']=le.transform(dataset['Fertiliser'])

In [80]:
dataset

,Fertiliser,N(%),P(%),K(%)
0,0,25.0,0,0
1,2,20.5,0,0
2,3,82.2,0,0
3,4,25.0,0,0
4,15,46.0,0,0
5,1,20.0,20,0
6,5,16.0,46,0
7,6,0.0,32,0
8,11,0.0,30,0
9,12,0.0,16,0


In [81]:

# selecting the features and labels
X = dataset.iloc[:, :-1].values
Y = dataset.loc[:, ['Fertiliser']]

print(X)
print(Y)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# splitting the model into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.30)


[[ 0.  25.   0. ]
 [ 2.  20.5  0. ]
 [ 3.  82.2  0. ]
 [ 4.  25.   0. ]
 [15.  46.   0. ]
 [ 1.  20.  20. ]
 [ 5.  16.  46. ]
 [ 6.   0.  32. ]
 [11.   0.  30. ]
 [12.   0.  16. ]
 [14.   0.  46. ]
 [ 7.   0.   0. ]
 [ 8.   0.   0. ]
 [ 9.  13.8  0. ]
 [10.   0.  56. ]
 [13.   0.   0. ]]
    Fertiliser
0            0
1            2
2            3
3            4
4           15
5            1
6            5
7            6
8           11
9           12
10          14
11           7
12           8
13           9
14          10
15          13


In [82]:
# from sklearn.ensemble import RandomForestRegressor
 
#  # create regressor object
# # regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# # fit the regressor with x and y data
# regressor.fit(X_train, y_train.values.ravel())
# predictions = regressor.predict(X_train)
# print("***********************************************************")


# print(regressor.score(X_train, y_train))

# print("***********************************************************")

***********************************************************
0.9799198757763975
***********************************************************


In [83]:
print(regressor.predict([[137,42,144]]))

[13.55]


In [84]:
sgd_classifier = SGDClassifier(tol=-np.infty, loss='log')
sgd_classifier=sgd_classifier.fit(X_train, y_train)

predictions = sgd_classifier.predict(X_train)
print("***********************************************************")
print("Accuracy = " + str(accuracy_score(y_train.values.ravel(), predictions)))
print("***********************************************************")

***********************************************************
Accuracy = 0.45454545454545453
***********************************************************


C:\Users\PRAMILA\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [92]:
pickle.dump(sgd_classifier, open('sgd_model.pkl','wb'))

In [93]:
results = pd.DataFrame(sgd_classifier.predict_proba([[100,42,144]]).transpose())
results.rename(columns={0: "score"}, inplace=True)

results.reset_index(level=0, inplace=True)
results.sort_values(by="score", ascending=False, inplace=True, ignore_index=True)



In [94]:
results

,index,score
0,9,5.000000e-01
1,10,5.000000e-01
2,4,1.342109e-306
3,0,0.000000e+00
4,1,0.000000e+00
5,2,0.000000e+00
6,3,0.000000e+00
7,5,0.000000e+00
8,6,0.000000e+00
9,7,0.000000e+00


In [87]:
total=results['score'].isnull().sum()
if total==len(results):
    print("no fertilizer needed")

In [89]:
for i in range(len(results)):
    if results.loc[i, "score"] > 0.001:
        print(str(labels[results.loc[i, "index"]]) + " (" + str(round(100 * results.loc[i, "score"])) + "%)")



Potassium nitrate (50%)
Potassium polyphosphate (50%)
